In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import time
import os
import math
import random
from scipy.stats import ttest_ind

from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [165]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    backward_lags = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)
    forward_lags  = pd.concat([data[var].shift(-1*lag*tau).rename(f'{var}(t+{lag*tau})') for lag in range(1,num_lags+1) for var in data.columns], axis=1)
    block = pd.concat([backward_lags, forward_lags], axis=1)

    return block

In [166]:
def get_valid_lags(all_lags, target, gap_radius, Tp):
    
    system_variables = []#'R', 'C1', 'C2', 'P1', 'P2'
    
    valid_lags = all_lags.copy()
    
    # If Tp = 0, remove [-gap_radius, gap_radius] lags of target variable from valid_lags
    if Tp == 0:
        for r in range(-gap_radius, gap_radius+1):
            if r < 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t{r})']
            elif r == 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t-{r})']
            elif r > 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t+{r})']
            
    # If Tp != 0, remove sgn(Tp)*[1,2r+1] lags of all variables from valid lags
    elif Tp != 0:
        sgn_Tp = int(math.copysign(1, Tp))
        lags_to_remove = [sgn_Tp * l for l in list(range(1,2*gap_radius+1 + 1))]
        for lag in lags_to_remove:
            for var in system_variables:
                if lag < 0:
                    valid_lags = [x for x in valid_lags if x != f'{var}(t{lag})']
                elif lag == 0:
                    valid_lags = [x for x in valid_lags if x != f'{var}(t-{lag})']
                elif lag > 0:
                    valid_lags = [x for x in valid_lags if x != f'{var}(t+{lag})']
            
    return valid_lags

In [167]:
def get_xmap_results_smap(block, target, embeddings, Tp, theta, lib, pred):
    '''Function to do exhaustive search of embeddings.'''
    
    def compute_rho(block, target, embedding, Tp, theta, lib, pred):
        xmap = SMap(dataFrame=block, target=target, columns=embedding, Tp=Tp, theta=theta, embedded=True, lib=lib, pred=pred, noTime=True)
        rho = xmap['predictions'][['Observations', 'Predictions']].corr().iloc[0,1]
        return embedding, xmap['predictions'], rho

    xmap_results = pd.DataFrame(columns=['embedding', 'rho'])
    xmap_results = Parallel(n_jobs=-1)(delayed(compute_rho)(block, target, embedding, Tp, theta, lib, pred) for embedding in embeddings)
    xmap_results = pd.DataFrame(xmap_results, columns=['embedding', 'result', 'rho'])
    xmap_results = xmap_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    return xmap_results

In [168]:
# Multiview Cross-Mapping Function

def MVCM(block, target, xmap_results, Tp, gap_radius, theta, lib, pred, E, k, self_weight):
    
    # Get lib and pred indices, adjusted to match pyEDM
    lib_start, lib_end = map(int, lib.split())
    pred_start, pred_end = map(int, pred.split())
    lib_start -= 1; lib_end -= 1
    pred_start -= 1; pred_end -= 1
    
    if Tp > 0:
        pred_end += Tp
    elif Tp < 0:
        pred_start -= -1 * Tp
    
    # If k > number of system views, return NaNs as the filtered timeseries
    if k > len(xmap_results):
        filtered_timeseries = pd.DataFrame([np.nan] * len(xmap_results.loc[0,'result']['Predictions']))
        return filtered_timeseries
    
    filter_input = pd.DataFrame()
    filter_input = pd.concat([xmap_results.loc[i,'result']['Predictions'] for i in range(0,k)], axis=1)
    filter_input.index = block.loc[pred_start:pred_end,:].index
    
    self = block.loc[pred_start:pred_end,f'{target}(t-0)']
    self.index = range(pred_start,pred_end+1)
    filter_input['self'] = self
    filter_input['vals_to_avg'] = filter_input.apply(lambda row: row.tolist(), axis=1)
    
    # Get weights based on cross-map skill of embeddings
    weights = xmap_results.loc[:k-1,'rho'].tolist()
    weights = [x if x >= 0 else 0 for x in weights]                  # Make negative weights 0
    
    if np.sum(weights) > 0:
        weights = [(1 - self_weight/100)*(weight/np.sum(weights)) for weight in weights]
    else:
        weights = [(1 - self_weight/100)*(1/len(weights)) for weight in weights]
    
    weights = weights + [self_weight/100]
    filter_input['weights'] = [weights] * len(filter_input)
    
    # Get filtered values
    vals_to_avg = np.array(filter_input['vals_to_avg'].tolist())
    weights = np.array(filter_input['weights'].tolist())

    filter_input['filtered_points'] = np.nansum(vals_to_avg * weights, axis=1)
    
    filtered_timeseries = filter_input[['filtered_points']].copy()
    
    # Make sure filtered values are positive
    filtered_timeseries[filtered_timeseries<0] = 0
    
    return filtered_timeseries

In [169]:
def optimize_parameters_MVCM(block, target, all_xmap_results, Tp, gap_radius, lib, pred, E_list, k_list, theta_list):
    
    # Get lib and pred indices, adjusted to match pyEDM
    lib_start, lib_end = map(int, lib.split())
    pred_start, pred_end = map(int, pred.split())
    lib_start -= 1; lib_end -= 1
    pred_start -= 1; pred_end -= 1
    
    # Optimize parameters using a self_weight of 0 until a self_weight is chosen at the end
    self_weight = 0
    
    # Choose the E, k, and theta that give the best multiview cross-map prediction of the observed data with a self_weight of 0

    xmap_results_dict = {}
    
    # Get multiview cross-map predictions for E, k, and theta combinations
    mvcm_results = pd.DataFrame(columns=['E', 'k', 'theta', 'rho', 'xmap_results', 'noisy_and_filtered'])
    
    total_iterations = len(list(product(E_list, theta_list, k_list)))
    #with tqdm(total=total_iterations) as pbar:
    for E, theta in product(E_list, theta_list):

        # Get random embeddings and their cross-map skill
        xmap_results = {k: v for k, v in all_xmap_results.items() if (k.split('_')[0] == target) & 
                                                            (k.split('_')[1] == lib) &
                                                            (k.split('_')[2] == str(E)) &
                                                            (k.split('_')[3] == str(theta))}
        key = list(xmap_results.keys())[0]
        xmap_results = xmap_results[key]
        xmap_results_dict['{0}_{1}'.format(E, theta)] = xmap_results

        # Get multiview cross-map predictions for ks in k_list 
        for k in k_list:

            filtered = MVCM(block, target, xmap_results_dict[f'{str(E)}_{str(theta)}'], Tp, gap_radius, theta, lib, pred, E, k, self_weight)

            # Align indices of noisy target with indices of filtered_timeseries
            noisy_target = block.loc[pred_start:pred_end,f'{target}(t-0)']
            noisy_and_filtered = pd.concat([noisy_target, filtered], axis=1)
            noisy_and_filtered.columns = [f'noisy_{target}', f'filtered_{target}']
            rho = noisy_and_filtered.corr().iloc[0,1]
            mvcm_results.loc[len(mvcm_results)] = [E, k, theta, rho, xmap_results, noisy_and_filtered]
                #pbar.update(1)

    mvcm_results = mvcm_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    E = int(mvcm_results.loc[0,'E'])
    k = int(mvcm_results.loc[0,'k'])
    theta = int(mvcm_results.loc[0,'theta'])
    
    return E, k, theta, mvcm_results

In [170]:
HAB_data = pd.read_csv('Data/data_w_gaps.csv', index_col=0).iloc[304:612]
HAB_data.columns = HAB_data.columns.str.replace(' ', '_')

# Put columns in alphabetical order
sorted_columns = sorted(HAB_data.columns)
HAB_data = HAB_data[sorted_columns]

# Make indices integers and save mapping to dates
#date_to_int_map = {i: date for i, date in enumerate(HAB_data.index)}
#HAB_data.index = range(len(HAB_data))
HAB_data.index = HAB_data.index.astype(int)

target = 'Avg_Chloro_(mg/m3)'

#HAB_data = HAB_data.drop(['Nitrite_(uM)','Nitrate_(uM)'],axis=1)

HAB_data = HAB_data[['Avg_Chloro_(mg/m3)','Nitrate_(uM)','Nitrite_(uM)','Phosphate_(uM)','Silicate_(uM)']]
HAB_data = HAB_data.drop('Nitrite_(uM)',axis=1)

block = get_block(HAB_data, num_lags=50, tau=1)
block
#HAB_data

,Avg_Chloro_(mg/m3)(t-0),Nitrate_(uM)(t-0),Phosphate_(uM)(t-0),Silicate_(uM)(t-0),Avg_Chloro_(mg/m3)(t-1),Nitrate_(uM)(t-1),Phosphate_(uM)(t-1),Silicate_(uM)(t-1),Avg_Chloro_(mg/m3)(t-2),Nitrate_(uM)(t-2),...,Phosphate_(uM)(t+48),Silicate_(uM)(t+48),Avg_Chloro_(mg/m3)(t+49),Nitrate_(uM)(t+49),Phosphate_(uM)(t+49),Silicate_(uM)(t+49),Avg_Chloro_(mg/m3)(t+50),Nitrate_(uM)(t+50),Phosphate_(uM)(t+50),Silicate_(uM)(t+50)
time (UTC),,,,,,,,,,,,,,,,,,,,,
2128,1.73,0.57,0.17,2.6,NaN,NaN,NaN,NaN,NaN,NaN,...,0.13,1.2,1.33,0.18,0.17,1.8,1.07,0.24,0.20,3.9
2135,2.20,1.70,0.29,5.5,1.73,0.57,0.17,2.6,NaN,NaN,...,0.17,1.8,1.07,0.24,0.20,3.9,2.04,2.38,0.36,6.5
2142,0.57,0.00,0.16,3.0,2.20,1.70,0.29,5.5,1.73,0.57,...,0.20,3.9,2.04,2.38,0.36,6.5,1.14,0.59,0.25,3.2
2149,0.70,0.21,0.17,5.1,0.57,0.00,0.16,3.0,2.20,1.70,...,0.36,6.5,1.14,0.59,0.25,3.2,1.52,0.44,0.19,2.8
2157,0.74,0.26,0.16,5.4,0.70,0.21,0.17,5.1,0.57,0.00,...,0.25,3.2,1.52,0.44,0.19,2.8,1.85,0.28,0.18,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4250,0.90,0.39,0.33,2.5,0.99,0.00,0.26,2.9,0.80,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4256,0.84,0.00,0.28,2.4,0.90,0.39,0.33,2.5,0.99,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4263,0.83,0.00,0.23,2.1,0.84,0.00,0.28,2.4,0.90,0.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    backward_lags = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)
    forward_lags  = pd.concat([data[var].shift(-1*lag*tau).rename(f'{var}(t+{lag*tau})') for lag in range(1,num_lags+1) for var in data.columns], axis=1)
    block = pd.concat([backward_lags, forward_lags], axis=1)
    
    return block

In [172]:
def ccm(interaction, block, E_list, tau_list, theta_list, Tp, sample=50, sig=0.05):
    print(interaction)
    lib = f'1 {len(block)}'
    
    # Get dataframe with two species of interest
    A = interaction[0]; B = interaction[1]
    df = block[[f'{A}(t-0)', f'{B}(t-0)']]
    
    driver = f'{A}(t-0)'
    
    E_tau_theta_results = pd.DataFrame(columns = ['E', 'tau', 'theta', 'rho'])
    for E, tau, theta in list(product(E_list, tau_list, theta_list)):
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho = c.corr().iloc[0,1]
        E_tau_theta_results.loc[len(E_tau_theta_results)] = [E, tau, theta, rho]
    E_tau_theta_results = E_tau_theta_results.apply(pd.to_numeric, errors='coerce')

    # Assign E, tau, and theta to be the optimal E, tau, and theta
    ccm_value = E_tau_theta_results['rho'].max()
    E = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'E'].item())
    tau = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'tau'].item())
    theta = int(E_tau_theta_results.loc[np.where(E_tau_theta_results.rho==ccm_value),'theta'].item())
        
    # Get convergence p-value
    convergence_p_value = get_convergence_p_value(block, sample, A, B, E, Tp, tau, theta)

    # Preparing Output
    output = {
        'target (driver)': A,
        'lib (driven)': B,
        'E': E,
        'tau': tau,
        'theta': theta,
        'E_tau_theta_results': E_tau_theta_results,
        'ccm_value': ccm_value,
        'convergence_p_value': convergence_p_value,
        'correlation': df.corr().iloc[0,1]
    }

    return output

def get_convergence_p_value(df, sample, A, B, E, Tp, tau, theta):
    # Get convergence p-value for CCM (one-tailed t-test on cross-map values using 20% and 50% library sizes)
    # H0: μ_20% ≥ μ_50%
    # HA: μ_20% < μ_50%
    # If p < 0.05, the 20% library size trials have a rho that is significantly smaller than the 50% library trials  
    
    libsize1 = int(np.ceil(df.shape[0]/5))   # 20% of the full library size
    libsize2 = int(np.ceil(df.shape[0]/2))   # 50% of the full library size
    
    max_iterations = 10 * sample
    
    # Get list of rhos for libsize1
    rhos1 = []; iteration_count = 0
    while len(rhos1) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize1, len(df))
        library = [start - libsize1, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho1 = c.corr().iloc[0,1]
        if not np.isnan(rho1):
            rhos1.append(rho1)
        iteration_count += 1
        
    # Get list of rhos for libsize2
    rhos2 = []; iteration_count = 0
    while len(rhos2) < sample and iteration_count < max_iterations:
        start = np.random.randint(libsize2, len(df))
        library = [start - libsize2, start]
        data_subset = df.iloc[library[0]:library[1]]
        lib = f'{library[0]+1} {library[1]+1}'
        driver = f'{A}(t-0)'
        driven_embedded = [f'{B}(t{i})' if i < 0 else f'{B}(t-{i})' for i in range(E * tau, 1)]
        driven_embedded = driven_embedded[::tau][:E]
        c = SMap(dataFrame=block, target=driver, columns=driven_embedded, embedded=True, Tp=Tp, theta=theta, lib=lib, pred=lib, noTime=True)
        c = c['predictions'][['Observations', 'Predictions']]
        rho2 = c.corr().iloc[0,1]
        if not np.isnan(rho2):
            rhos2.append(rho2)
        iteration_count += 1
    
    convergence_t_stat, convergence_p_value = ttest_ind(rhos1, rhos2, alternative='less')
    
    return convergence_p_value


In [173]:
E_list = range(2,13)
tau_list = [-1,-2,-3]
theta_list = [0,0.1,0.5,1,2,3,4,5,6,7,8,9]
Tp = 0
exclusion_radius = 0

all_ccm_results = pd.DataFrame()
interactions = list(permutations(HAB_data.columns.tolist(),2))
target_interactions = [pair for pair in interactions if target in pair]

interaction = target_interactions[0]
print(f'There are {len(target_interactions)} interactions')

results = Parallel(n_jobs=-1)(
    delayed(ccm)(interaction, block, E_list, tau_list, theta_list, Tp) for interaction in target_interactions)
results_df = pd.DataFrame(results)

There are 6 interactions
('Avg_Chloro_(mg/m3)', 'Phosphate_(uM)')
('Avg_Chloro_(mg/m3)', 'Silicate_(uM)')
('Nitrate_(uM)', 'Avg_Chloro_(mg/m3)')
('Avg_Chloro_(mg/m3)', 'Nitrate_(uM)')
('Silicate_(uM)', 'Avg_Chloro_(mg/m3)')
('Phosphate_(uM)', 'Avg_Chloro_(mg/m3)')


In [176]:
# Get CCM results that show convergence (convergence p-value < 0.05)

ccm_cutoff = 0.1

significant_results = results_df[results_df.convergence_p_value<0.05]
significant_results = significant_results.sort_values(by='ccm_value', ascending=False)
significant_results = significant_results[['target (driver)', 'lib (driven)', 'E', 'tau', 'theta', 'ccm_value']].reset_index(drop=True)

display(significant_results[significant_results.ccm_value>ccm_cutoff])

# Choose system variables where the CCM value to or from the target is > ccm_cutoff
system_variables = significant_results[significant_results.ccm_value > ccm_cutoff]
system_variables = system_variables[['target (driver)', 'lib (driven)']].values.flatten().tolist()
system_variables = list(set(system_variables))
print('system variables: ')
display(sorted(system_variables))

,target (driver),lib (driven),E,tau,theta,ccm_value
0,Silicate_(uM),Avg_Chloro_(mg/m3),7,-3,4,0.467497
1,Avg_Chloro_(mg/m3),Nitrate_(uM),5,-3,9,0.355231


system variables: 


['Avg_Chloro_(mg/m3)', 'Nitrate_(uM)', 'Silicate_(uM)']

In [177]:
def get_valid_lags_tau(block, target, Tp, tau, num_lags, exclusion_radius, system_variables):
    
    # Get lags of system variables
    system_variable_lags = []
    for var in system_variables:
        # Get forwards and backwards lag of the system variables
        var_backwards_lags = [f'{var}(t{i})' if i < 0 else f'{var}(t-{i})' for i in range(num_lags * tau, 1)]
        var_backwards_lags = var_backwards_lags[::tau][:num_lags]
        var_forwards_lags  = [f'{var}(t+{i})' for i in range(-(num_lags-1) * tau + 1)]
        var_forwards_lags  = var_forwards_lags[::tau][:num_lags-1]
        var_lags = var_backwards_lags + var_forwards_lags
        system_variable_lags = system_variable_lags + var_lags
    
    # Remove (t-0) lag of target variable from valid_lags
    valid_lags = [x for x in system_variable_lags if x != f'{target}(t-0)']

    # If Tp = 0, remove [-exclusion_radius, exclusion_radius] lags of target variable from valid lags
    if Tp == 0:
        for r in range(-exclusion_radius, exclusion_radius+1):
            if r < 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t{r})']
            elif r == 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t-{r})']
            elif r > 0:
                valid_lags = [x for x in valid_lags if x != f'{target}(t+{r})']
                    
    return valid_lags

#target = 'Planktothrix_rubescens'
system_variables = system_variables
Tp = 0
exclusion_radius = 6
num_lags = 2   # Use -3, 0, and +3 lags of each variable
tau = -3

valid_lags = get_valid_lags_tau(block, target, Tp, tau, num_lags, exclusion_radius, system_variables)
valid_lags

['Nitrate_(uM)(t-0)',
 'Nitrate_(uM)(t-3)',
 'Nitrate_(uM)(t+3)',
 'Silicate_(uM)(t-0)',
 'Silicate_(uM)(t-3)',
 'Silicate_(uM)(t+3)']

In [178]:
random_embeddings = {}
for E in range(4,26):
    # Get random embeddings using valid lags
    embeddings = set()
    sample = 100000
    max_trials = 10000000
    trials = 0
    while len(embeddings) < sample and trials < max_trials:
        embedding = tuple(random.sample(valid_lags, E))
        sorted_embedding = tuple(sorted(embedding))
        if sorted_embedding not in embeddings:
            embeddings.add(sorted_embedding)
    trials += 1
    embeddings = [list(embedding) for embedding in embeddings]
    random_embeddings['{0}'.format((target, E, Tp, exclusion_radius))] = embeddings
    print(f'E = {E}, # embeddings = {len(embeddings)}')
    
with open('random_embeddings_HAB.pkl', 'wb') as file:
     pickle.dump(random_embeddings, file)

KeyboardInterrupt: 

In [ ]:
# Load HAB random embeddings
with open('random_embeddings_HAB.pkl', 'rb') as file:
    HAB_embeddings = pickle.load(file)

In [ ]:
# Create folder to store xmap results
folder = 'xmap results HAB 100000 random embeddings'
if not os.path.exists(folder):
    os.makedirs(folder)

In [ ]:
# Save HAB cross-mapping results

E_list = range(4,25)
theta_list = [1,5,9,15,25,35,45]
Tp = 0
exclusion_radius = 6
self_weight = 0  # self_weight = 0 for gap filling
lib = '1 396' #CHANGE TO LIBRARY SIZE
pred = '1 396'

total_iterations = len(E_list) * len(theta_list)

gapfill_results = {}
parameters = pd.DataFrame(columns=['target', 'noise_level', 'lib', 'pred', 'E', 'theta', 'k', 'rho'])
block = get_block(HAB_data, num_lags=50)

with tqdm(total=total_iterations) as pbar:
    for E, theta in product(E_list, theta_list):

        key = [key for key in HAB_embeddings.keys() if eval(key)[0] == target and eval(key)[1] == E and eval(key)[2] == Tp and eval(key)[3] == exclusion_radius]
        embeddings = HAB_embeddings[key[0]]

        xmap_results = get_xmap_results_smap(block, f'{target}(t-0)', embeddings, Tp, theta, lib, lib)

        file_path = os.path.join(folder, f'xmap_results_{target}_Tp_{Tp}_E_{E}_theta_{theta}.pkl')

        with open(file_path, 'wb') as f:
            pickle.dump(xmap_results, f)

        pbar.update(1)